# BackEnd for Vacuum Gauge Explorer
Contains the classes needed to operate the Vacuum Gauge explorer:
* **VGcontext**: Contains information common to all gauges in a fill e.g. BeamEnergy, BeamIntensity and Bunch Lengths
* **VGprobe**: Contains information specific to a probem, including location & model 
* **VGplot**: Contains methods needed to visualize 1 or more probes
* **VGcontrol**: Contrains the tools for stepping through gauges intelligently and applying analyzers to them 

In [ ]:
%run BackEnd_Plotters.ipynb
%run BackEnd_DataProcessing.ipynb
%run BackEnd_K-Neighbour.ipynb
%run BackEnd_Analytical_Classifiers.ipynb
import datetime
import csv
import matplotlib as mlt
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import Javascript
from IPython.display import clear_output
from temperature import Temperature
from joblib import dump, load
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
mlt.style.use("ggplot")
db = pytimber.LoggingDB()

class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'

# Fill Data Class
Stores and manipulates data relating to a fill, rather than a specific gauge

In [ ]:
class VGcontext:
    
    def __init__(self,fillNumber,model=None):
        self.fillNumber = fillNumber
        
    def populateAndCheckData(self):
        fill = db.getLHCFillData(self.fillNumber)
        if fill is None:
            return False
        self.t1 = fill['startTime']
        self.t2 = fill['endTime']
        date1 = datetime.datetime.fromtimestamp(self.t1)
        date2 = datetime.datetime.fromtimestamp(self.t2)
        timeBetween = date2 - date1
        if timeBetween.seconds<(60*60*5):
            return False
        
        self.beamNumber=1
        beamEnergyVariable='LHC.BOFSU:OFSU_ENERGY'
        beamEnergy=db.get(beamEnergyVariable, self.t1, self.t2)
        if not all(beamEnergy): #True == Non-zeros present
            return False
        self.beamEnergy = beamEnergy[beamEnergyVariable]
        
        beamIntensityVariable= 'LHC.BCTFR.A6R4.B'+str(self.beamNumber)+':BEAM_INTENSITY'
        beamIntensity=db.get(beamIntensityVariable, self.t1, self.t2)
        if not all(beamIntensity): #True == Non-zeros present
            return False
        self.beamIntensity = beamIntensity[beamIntensityVariable]
        
        beamBunchVariable='LHC.BQM.B'+str(self.beamNumber)+':BUNCH_LENGTH_MEAN'
        beamBunchLengths=db.get(beamBunchVariable, self.t1, self.t2)
        if not all(beamBunchLengths): #True == Non-zeros present
            return False
        self.beamBunchLengths = beamBunchLengths[beamBunchVariable]
        
        return True
        
    def __str__(self):
        return "Start Time: %s\nEnd Time: %s\nbeamIntensity: %s"%(datetime.datetime.fromtimestamp(self.t1),
                                                                  datetime.datetime.fromtimestamp(self.t2),
                                                                  len(self.beamIntensity[0]))


# VG Probe Class
Store and manipulate data pertaining to specific gauges

In [ ]:
class VGprobe(object):
    
    def __init__(self,fillNumber=None, chosen_locations = None):
        self._selectData(fillNumber, chosen_locations)
    
    def pick_fill_number(self,fillNumber=None):
        if fillNumber is None:
            try:
                maxFillNumber = db.getLHCFillData(None)['fillNumber']
            except:
                maxFillNumber = 7492
            print("Valid Fill Numbers: 0 - %d (inclusive)" % (maxFillNumber))
            fillNumber = input("Please select a fill number >>>")
            while(True):
                try:
                    fillNumber = int(fillNumber)
                    if(fillNumber < 0 or fillNumber > maxFillNumber):
                        print("Please enter a fill number in the correct range")
                        input("Select a fill number >>>")
                        continue
                    break
                except:
                    print("Please enter a valid integer")
                    input("Select a fill number >>>")
        return fillNumber
    
    def pick_model(self):
        model = "VGPB"
        return model
    
    def pick_locations(self,model):
        raw_data = db.search(model+'%')
        extracted_data = []
        for gauge_name in raw_data:
            parts = gauge_name.split('.')
            try:
                segment = parts[2]
            except:
                continue
            if bool(re.search('\d\w\d', segment)):
                extracted_data.append(segment) #Match!
                
        # Returns Unique Array containing all Segments
        print("Possible locations:")
        possible_locations = np.unique(np.array(extracted_data)) 
        display(possible_locations)
        
         # SELECTION OF LOCATIONS TO PLOT  
        print("Select a segment by number. Example: 3L17 \nOr 'a' to select all (slow)")
        resp = input(">>>")
        chosen_locations = []
        while(True):
            if resp.lower().strip() == "a":
                print("Selecting ALL Locations (may take some time)")
                chosen_locations = possible_locations
                break
            elif (re.match("\d+[L|R]\d+",resp.upper())):
                if(resp.upper() not in possible_locations):
                    print("That is not one of the possible locations, please try again")
                    resp = input(">>>")
                    continue
                print("Select a segment by number. Example: 3L17")
                print("Type 'x' to leave")
                chosen_locations.append(resp.upper())
                print("Previously selected locations: {}".format(chosen_locations))
            elif(resp.strip().lower() == "x"):
                if(len(chosen_locations)==0):
                    print("Must have at least one location selected")
                else:
                    break
            elif(re.match("\d+[L|R]",resp.upper())):
                for location in possible_locations:
                    if re.match(resp.upper(),location):
                        chosen_locations.append(location)
                print("Previously selected locations: {}".format(chosen_locations))
            else: 
                print("Invalid response, please try again (e.g. 1L1 or 8R22)")
            resp = input(">>>") 
        return chosen_locations

    
    def _selectData(self,fillNumber, chosen_locations, variable="VGI%.PR"):
        # >>>> Pick a fill number <<<<
        self.fillNumber = self.pick_fill_number(fillNumber)
        # >>>> END Pick a fill number <<<<

        # >>>> Pick a Model <<<<
        self.model = self.pick_model()
        # >>>> END Pick a Model <<<<
        
        # >>>> Pick multiple locations <<<<
        if chosen_locations is None:
            chosen_locations = self.pick_locations(self.model)
               
        self.locations = chosen_locations
        # >>>> END Pick multiple locations <<<<
        return True

# Vacuum Gauge Plot
Plot vacuum gauges from their context and their associated probe data

In [ ]:
class VGplot(object):
    
    def __init__(self,data,context):
        self.data = data
        self.context = context
    
    def plotAllLocationsOnOnePlot(self, verbose = False, limit = 6):
        
        def plot_gauge_data(self,f,axes_tuple):
            # >>>> Plot Gauge Data <<<<
            ax1,ax2,ax3 = axes_tuple
            
            # Stylize Vacuum Gauge plots
            ax1.set_title("Intensity versus Pressure for Fill %s"%(self.data.fillNumber))
            ax1.set_ylabel('Pressures')
            ax1.set_yscale('log')
            ax1.legend(ncol=int(math.ceil(len(self.data.locations)/4.0)),fontsize='xx-small',loc='upper left')

            # Stylize Beam Intensity
            ax1_1 = ax1.twinx()
            ax1_1.set_ylabel('Charges')
            ax1_1.plot(self.context.beamIntensity[0], self.context.beamIntensity[1], label=
                       'LHC.BCTFR.A6R4.B'+str(self.context.beamNumber)+':BEAM_INTENSITY',color='b')
            ax1_1.legend(fontsize='xx-small',loc='upper right')
            ax1_1.grid(False)
            pytimber.set_xaxis_date()

            ax2.set_ylabel('Pressures (mbar)')
            ax2.legend(ncol=int(math.ceil(len(self.data.locations)/4.0)),fontsize='xx-small',loc='upper left')
                
            # Stylize Beam Intensity
            ax2_1 = ax2.twinx()
            ax2_1.set_ylabel('Charges')
            ax2_1.plot(self.context.beamIntensity[0], self.context.beamIntensity[1], label=
                       'LHC.BCTFR.A6R4.B'+str(self.context.beamNumber)+':BEAM_INTENSITY',color='b')
            ax2_1.legend(fontsize='xx-small',loc='upper right')
            ax2_1.grid(False)
            pytimber.set_xaxis_date()

            # Make the y-axis label, ticks and tick labels match the line color.
            ax3.set_title('Beam Energy versus Bunch Length')
            ax3.plot(self.context.beamEnergy[0], self.context.beamEnergy[1],label='LHC.BOFSU:OFSU_ENERGY', color='r')
            ax3.set_ylabel('GeV')
            ax3.set_xlim(np.min(self.context.beamIntensity[0]),np.max(self.context.beamIntensity[0]))
            ax3.legend(fontsize='xx-small',loc='upper left')
            
            ax3_1 = ax3.twinx()
            ax3_1.plot(self.context.beamBunchLengths[0], self.context.beamBunchLengths[1],label='LHC.BQM.B'+str(self.context.beamNumber)+':BUNCH_LENGTH_MEAN', color='b')
            ax3_1.set_ylabel('s')
            ax3_1.legend(fontsize='xx-small',loc='upper right')
            ax3_1.grid(False)
            
            date=str(datetime.datetime.now()).replace(' ','_')
            plt.tight_layout()
            f.canvas.draw()
            f.show()
            return f, (ax1,ax2,ax3)
            # >>>> END Plot Gauge Data <<<<
        
        if self.data is None:
            return

        # >>>> Retrieve Gauge Data <<<<
        progessBar = IntProgress(min=0, max=len(self.data.locations)) # instantiate the bar
        display(progessBar)

        # Create empty Dataframe to transfer re-formatted data to
        df2 = pd.DataFrame()
        # For initally populating df
        df2IsInitialised = False
       
        
        # Plot to work on (axes belong to f)
        self.figures = {}
        self.axes = {}
        fig_count = 0
        self.figures["fig{0}".format(fig_count)],\
        self.axes["axes{0}".format(fig_count)] = plt.subplots(3,
                                                              sharex=True,
                                                              gridspec_kw = {'height_ratios':[13,13,5]},
                                                              figsize=(10, 6))
        
        self.gauge_ids = []
        count = 0
        # New figure for each location
        # All gauges at a specific location
        for location in self.data.locations:
             # Specific gauge to look for
            gauge_id = self.data.model+'.%.'+location+'.%.PR'
            # Returns a dictionary
            VG = db.get(gauge_id, self.context.t1, self.context.t2) 
            if (VG == {}): #empty
                print("Skipping %s as there is no data" % (gauge_id))
                continue

            # Generate dataframe from the dictionary, give it named indices
            df = pd.DataFrame.from_dict(VG,columns=['timeStamp','pressure'],orient='index')
            

            for gauge_id, readings in VG.items():
                # Narrow down exact ID
                self.gauge_ids.append(gauge_id)

                # Expand out the timeStamp and pressure cells (lists) into a Series
                timeStamp = pd.Series(readings[0])
                pressure_readings = pd.Series(readings[1])
                
                # Diagnostic
                if verbose:
                    print(">>>> DIAGNOSTIC <<<< \n {} Info: \ntimeStamp shape: {} pressure shape: {}".format(gauge_id,
                                                                                                  timeStamp.shape,pressure_readings.shape))
                
                # Add to DataFrame (unequal lengths are filled with NaN)
                if not df2IsInitialised:
                    df2[gauge_id+":time"] = timeStamp
                    df2[gauge_id+":pressure"] = pressure_readings 
                    df2IsInitialised = True
                else:
                    temp1 = pd.DataFrame()
                    temp1[gauge_id+":time"] = timeStamp
                    df2 = pd.concat([df2.reset_index(drop=True) ,temp1.reset_index(drop=True) ],sort=False,axis=1)
                    temp2 = pd.DataFrame()
                    temp2[gauge_id+":pressure"] = pressure_readings
                    df2 = pd.concat([df2.reset_index(drop=True) ,temp2.reset_index(drop=True) ],sort=False,axis=1)
                    
                self.axes["axes{0}".format(fig_count)][0].plot(df2[gauge_id+":time"],df2[gauge_id+":pressure"],label=gauge_id)
                self.axes["axes{0}".format(fig_count)][1].plot(df2[gauge_id+":time"],df2[gauge_id+":pressure"],label=gauge_id)
                count += 1
                if count >= limit:
                    if verbose:
                        print("Count %d exceeds limit %d"%(count,limit))
                    count = 0
                    self.figures["fig{0}".format(fig_count)],self.axes["axes{0}".format(fig_count)] = plot_gauge_data(self,
                                                                        self.figures["fig{0}".format(fig_count)],
                                                                        self.axes["axes{0}".format(fig_count)])
                    fig_count += 1
                    # Reset to New Figure
                    self.figures["fig{0}".format(fig_count)],self.axes["axes{0}".format(fig_count)] = plt.subplots(3, sharex=True, 
                                                                                                                   gridspec_kw = {'height_ratios':[13,13,5]},
                                                                                                                   figsize=(10, 6))
            progessBar.value += 1
        plot_gauge_data(self,
                        self.figures["fig{0}".format(fig_count)],
                        self.axes["axes{0}".format(fig_count)])
        # >>>> END Retrieve Gauge Data <<<< 
        
        
    def savePlot(self):
        for key,item in self.figures.items():
            figure = item
            locs = sort(self.data.locations)
            folder = os.path.join(os.getcwd(),"data","figures","fill"+str(self.data.fillNumber))
            fileName="Fill%s-Loc%s-To-Loc%s-%s.svg"%(self.data.fillNumber,locs[0],locs[-1],key)
            if not os.path.exists(folder):
                os.makedirs(folder)
            if not os.path.isfile(os.path.join(folder,fileName)):
                figure.savefig(os.path.join(folder,fileName), format='svg',dpi=1000, facecolor='w', edgecolor='w',
                orientation='portrait',
                transparent=False, bbox_inches=None, pad_inches=0.1,)
                print("Successfully saved to %s"%(fileName))
            else:
                print("File: %s/%s \n Already exists"%(folder,fileName))

In [ ]:
class VGcontrol:
            
    def __init__(self, verbose = False):
        try:
            self.maxFillNo = db.getLHCFillData(None)['fillNumber']
        except:
            self.maxFillNo = 7492
        self.fillNo = None
        self.bias = None
        self.teller = 0
        self.try_to_classify = False
        self.save_locations = False
        self.saved_locations = None
        self.classification_count = 0
        
        self.main(verbose)
        
    def save_result(self, probe_id, fill_no, skipSteepness=True):
        print("Append %s to CSV file by selecting classification. \n1 is coupled, 0 is not coupled. 2 to skip."%probe_id)
        resp = input(">>>")
        coupling = None
        while True:
            try:
                resp = int(resp.strip())
            except:
                print("Please type 0 or 1 as integer values")
                resp = input(">>>")
                continue
            if resp == 0 or resp == 1:
                coupling = "NORMAL" if resp == 0 else "COUPLED"
                break
            elif resp == 2:
                coupling = "UNDETERMINED"
                break
            else:
                print("Please type 0 or 1 as integer values")
                resp = input(">>>")
                
        steepness = "UNDEFINED"
        if not skipSteepness:
            print("Select a steepness.\n0: Low \n1: Medium\n2:High\n3:Unknown")
            resp = input(">>>")
            while True:
                try:
                    resp = int(resp.strip())
                except:
                    print("Please type an integer. 0: Low, 1: Medium, 2: High, 3: Unknown") 
                    resp = input(">>>")
                    continue
                if resp in range(0,4):
                    steepness = resp
                    break
                else:
                    print("Please select one of these 4 options. 0: Low, 1: Medium, 2: High, 3: Unknown") 
                    resp = input(">>>")
         
        with open('NewProbeCatalogue.csv','a') as fd:
            comment = "Auto-Generated using Explorer"
            writer = csv.writer(fd)

            row = [probe_id,steepness,coupling,probe_id.split('.')[0],comment,fill_no]
            writer.writerow(row)
                
    def pick_model(self):
        folder = os.path.join(os.getcwd(),"data","models","k_neighbours")
        files = os.listdir(folder)
        for i in range(0,len(files)):
            print("%s%d: %s%s" %(color.BOLD,i,color.END,files[i]))
        print("Type number of desired model to load it")
        while True:
            resp = input(">>>")
            try:
                file_name = files[int(resp)]
            except:
                continue
            model = load(os.path.join(folder,file_name))
            print("Model Loaded with %d neighbours"%model.n_neighbors)
            break
        return model, file_name
    
    def use_analyzer(self, gauge_id, fillNo, model, name, showPlot=True, verbose=True):
        hot_to_class = {0:"Normal Gauge Behaviour", 1: "Coupling Detected"}
        cuts = int(name.split('_')[-1].split('.')[0])
        fillNo = int(fillNo)
        file_name = "probe_%s_fill%d.p"%(gauge_id,fillNo)
        press_file_name = "pressure_levels_for_fill_%d_w_%d_cuts.p"%(fillNo,cuts)
        folder = os.path.join(os.getcwd(),'data','probes',gauge_id)
        if not os.path.exists(folder):
            os.makedirs(folder)
        if os.path.isfile(os.path.join(folder,file_name)):
            if verbose:
                print("Loading existing data for %s"%gauge_id)
            pgd = pickle.load(open(os.path.join(folder,file_name),"rb"))
            if os.path.isfile(os.path.join(folder,press_file_name)):
                pressure_levels = pickle.load(open(os.path.join(folder,press_file_name),"rb"))
            else:
                pressure_levels = level_data(pgd.pressure_readings[pgd.mask],cuts)
                pickle.dump(pressure_levels, open( os.path.join(folder,press_file_name),"wb"))
        else:
            if verbose:
                print("Saving data for %s"%gauge_id)
            pgd = processed_gauge_data(gauge_id, fillNo)
            pressure_levels = pgd.generate_data(cuts=cuts)
            if pressure_levels is None:
                print("No Data exists for {} at fill {}".format(gauge_id,fillNo))
                return None
            pickle.dump( pgd, open( os.path.join(folder,file_name), "wb" ))
            pickle.dump(pressure_levels, open( os.path.join(folder,press_file_name), "wb"))
        
        # Prediction
        reshaped_pressure_levels = pressure_levels.reshape(1,-1)
        print("Learned Prediction:")
        try:
            print(hot_to_class[model.predict(reshaped_pressure_levels)[0]])

            f1, ax1 = plot_levelled_data(gauge_id, pressure_levels, 
                                          pgd.time_readings[pgd.mask],
                                          pgd.pressure_readings[pgd.mask])
            f1.canvas.draw()
        except ValueError:
            print("Invalid value (NaN or Infinity) broke model")
            return
        
        print("Analytical Prediction:")
        try:
            f2,ax2,coupled = analytical_classifier(gauge_id,fillNo,showPlot=True)
            f2.canvas.draw()
        except UnboundLocalError:
            print("Could not fit analytical analyzer to {} in fill {}".format(gauge_id,fillNo))
            return None

        self.save_result(gauge_id,fillNo,skipSteepness=True)
            
    def main(self, verbose):
        def move_forward_one_fill(self):
            self.fillNo += 1 if self.fillNo <= self.maxFillNo else print("Reached furthest fill")

        def reload_current_fill(self):
            self.fillNo = self.fillNo

        def move_backward_one_fill(self):
            self.fillNo -= 1 if self.fillNo >= 1 else print("Reached earliest fill")

        def jump_to_specific_fill(self):
            self.fillNo = None

        def step_through_X_fills(self):
            resp = input("Pick a direction, a: back, d: forward")
            self.bias = None
            bias_options = {"a":-1,"d":1}
            while self.bias is None:
                resp = resp.strip().lower()
                if resp in bias_options:
                    self.bias = bias_options[resp]
                else:
                    resp = input("Pick a direction, a: back, d: forward")
            resp = input("Type the number of fills to go through")

            self.teller = 0
            while self.teller == 0:
                try:
                    resp = int(resp)
                except ValueError:
                    print("Please type a number")
                    resp = input("Type the number of fills to go through")
                    continue
                if (self.bias == 1 and resp < self.maxFillNo-self.fillNo) or (self.bias == -1 and resp < self.fillNo):
                    self.teller = resp
                    return
                else:
                    print("Chosen number would go out of bounds, try again")
                    resp = input("Type the number of fills to go through")

        def fix_locations(self):
            if self.saved_locations is not None:
                self.saved_locations, self.save_locations = None, False
            else:
                self.save_locations = True

        def apply_classifiers(self):
            self.try_to_classify=True
        
        info = """
        At Fill %s/%s
        s: reload current fill
        d: move forward one fill
        a: move backward one fill
        j: jump to a specific fill
        b: step through X fills in specified direction
        f: fix selected location until called again
        x: to quit this wizard
        z: analyze using Classifiers"""
        options = {"a":move_backward_one_fill,"s":reload_current_fill,"d":move_forward_one_fill,
                   "j":jump_to_specific_fill,"b":step_through_X_fills,"f":fix_locations,
                   "z":apply_classifiers}
        resp = "j"
        while True:
            # If in automatic step-through mode
            if self.bias != None and self.teller > 0:
                self.fillNo += self.bias
                self.teller -= 1
            else: # Normal
                resp = resp.strip().lower()
                if resp == "x":
                    print("Classified %d gauges"%self.classification_count)
                    break
                clear_output()
                if resp in options:
                    options[resp](self)
                else:
                    print("Invalid response, please try again")
                    print(info%(self.fillNo,self.maxFillNo))
                    resp = input(">>>")
                    continue

            data = VGprobe(self.fillNo, self.saved_locations)
            if self.save_locations:
                    self.saved_locations = data.locations
                    self.save_locations = False
                    print("Location fixed, select 'f' again to reset")
            
            self.fillNo = data.fillNumber
            context = VGcontext(self.fillNo)
            check = context.populateAndCheckData()
            if check:
                if verbose:
                    print("Data for fill %s: \n Loc: %s " % (self.fillNo, data.locations))
                    print("Context: \n %s" % context)
                    print("Check %s" % check)
                
                plot = VGplot(data,context)
                plot.plotAllLocationsOnOnePlot(verbose)
                plot.savePlot()
                
                if self.try_to_classify:
                    gauge_ids = plot.gauge_ids
                    hot_to_class = {0:'Normal Gauge Behaviour',1:'Coupling Detected!'}
                    model, model_name = self.pick_model()
                    for gauge_id in gauge_ids:
                        print("Using {} in analyzer".format(gauge_id))
                        self.use_analyzer(gauge_id, self.fillNo, model, model_name, showPlot=True)
                        self.classification_count += 1
                        break
                    self.try_to_classify=False
                    plt.close()
                    break
            else:
                print("Fill %s REJECTED "%self.fillNo)
            if self.teller == 0:
                print(info%(self.fillNo,self.maxFillNo))
                resp = input(">>>")
            


In [ ]:
controller = VGcontrol(verbose=False)